In [ ]:
# --- Load the previously merged panel dataset ---

import pandas as pd

panel = pd.read_csv("sustainability-economic-performance/data/processed/panel_full_unfiltered.csv")

# display the first 10 rows
panel.head(10)

In [ ]:
# how big is this panel ?
panel.shape, panel.columns

In [ ]:
# --- Create a table showing the distribution of all 192 countries ---

# keep only one row per country
countries_unique = panel[["Country Name", "Region", "Income Group"]].drop_duplicates()

# create the Region × Income Group table
region_income_table = pd.crosstab(
    countries_unique["Region"],
    countries_unique["Income Group"],
    margins = True,
    margins_name = "Total"
)

# remove axis labels so "Region" and "Income Group" don't appear inside the table
region_income_table.index.name = None
region_income_table.columns.name = None

# add a grey background to the total rows and columns
def highlight_total_row(row):
    if row.name == "Total":
        return ['font-weight: bold; background-color: #f0f0f0'] * len(row)
    else:
        return [''] * len(row)

def highlight_total_col(col):
    if col.name == "Total":
        return ['font-weight: bold; background-color: #f0f0f0'] * len(col)
    else:
        return [''] * len(col)

# build a nicer styled table
styled_192_table = (
    region_income_table
    .style
    .set_table_styles([
        # whole table border behavior
        {
            "selector": "table",
            "props": [
                ("border-collapse", "collapse"),
                ("border", "1px solid #aaaaaa")
            ],
        },
        # header cells (top row: income groups)
        {
            "selector": "th.col_heading",
            "props": [
                ("background-color", "#f5f5f5"),
                ("font-weight", "bold"),
                ("border", "1px solid #aaaaaa"),
                ("padding", "6px"),
                ("text-align", "center")
            ],
        },
        # row labels (left side: regions + Total)
        {
            "selector": "th.row_heading",
            "props": [
                ("background-color", "#f5f5f5"),
                ("font-weight", "bold"),
                ("border", "1px solid #aaaaaa"),
                ("padding", "6px"),
                ("text-align", "left")
            ],
        },
        # corner cell (top-left)
        {
            "selector": "th.blank",
            "props": [
                ("background-color", "#f5f5f5"),
                ("border", "1px solid #aaaaaa")
            ],
        },
        # data cells
        {
            "selector": "td",
            "props": [
                ("border", "1px solid #cccccc"),
                ("padding", "6px"),
                ("text-align", "center")
            ],
        },
    ])
    .apply(highlight_total_row, axis=1)  # style Total row
    .apply(highlight_total_col, axis=0)  # style Total column
)

styled_192_table

In [ ]:
# --- Use matplotlib to convert this table to a pdf ---

import matplotlib.pyplot as plt
import textwrap

df = region_income_table.copy()
n_rows, n_cols = df.shape

# --- Wrap long labels ---

def wrap_text(text, width):
    """Insert line breaks every `width` characters (on nearest space)."""
    return "\n".join(textwrap.wrap(str(text), width=width))

# Wrap long region names (row index)
wrapped_index = [wrap_text(idx, width=23) for idx in df.index]

# Wrap long column headers (income groups)
wrapped_columns = [wrap_text(col, width=12) for col in df.columns]

# create the figure
fig, ax = plt.subplots(figsize=(15, 10))
ax.axis("off")

# build the table
tbl = ax.table(
    cellText=df.values,
    rowLabels=wrapped_index,      # regions + "Total"
    colLabels=wrapped_columns,    # income groups + "Total"
    cellLoc="center",
    rowLoc="center",
    loc="center",
    colWidths=[0.11] + [0.11] * (n_cols - 1),  # first col + others
)

# adjust cell heights
for (row, col), cell in tbl.get_celld().items():
    if row == 0:        # header row
        cell.set_height(0.06)
    else:               # body rows
        cell.set_height(0.07)

# 3. font size
tbl.auto_set_font_size(False)
tbl.set_fontsize(12)

# 4. style cells
for (row, col), cell in tbl.get_celld().items():
    # borders
    cell.set_edgecolor("black")
    cell.set_linewidth(1)

    # header row (income groups)
    if row == 0:
        cell.set_text_props(weight="bold")
        cell.set_facecolor("#f5f5f5")

    # row labels (regions + "Total") live in column -1 in matplotlib's table
    if col == -1:
        cell.set_text_props(weight="bold")
        cell.set_facecolor("#f5f5f5")

    # total column OR Total row in bold + light grey
    if col == n_cols - 1 or row == n_rows:
        cell.set_text_props(weight="bold")
        cell.set_facecolor("#f0f0f0")

plt.tight_layout()

# save to PDF under results/country_distribution
output_path = "sustainability-economic-performance/results/country_distribution/region_income_table_192.pdf"
plt.savefig(output_path, bbox_inches="tight")
plt.show()

output_path

In [ ]:
# --- Compute data availability per country ---

# create a table showing how many entries there is per country, and what percentage of these entries are non-missing
country_stats = (
    panel
    .groupby(["Country Name", "Country Code", "Region", "Income Group"])
    .agg(
        total_points=("Value", "size"),
        non_missing=("Value", lambda x: x.notna().sum())
    )
    .reset_index()
)

country_stats["share_non_missing"] = (
    country_stats["non_missing"] / country_stats["total_points"]
)

# rename the columns to clearer names
country_stats = country_stats.rename(columns={
    "total_points": "Total Values",
    "non_missing": "Non Missing",
    "share_non_missing": "% of non-missing"
})

# display all 192 countries
country_stats.head(192)

In [ ]:
# take a look at the overall distribution of the panel
country_stats["% of non-missing"].describe()

In [ ]:
# sort the dataset by data availability
country_stats_sorted = country_stats.sort_values(
    "% of non-missing", ascending=False
)

# display the 50 countries with the most data available
country_stats_sorted.head(50)

In [ ]:
# filter only South Asia
south_asia = country_stats[country_stats["Region"] == "South Asia"].copy()

# select only the columns we want to display
south_asia_table = south_asia[[
    "Country Name",
    "Country Code",
    "Income Group",
    "% of non-missing"
]]

# sort countries by data availability (highest first)
south_asia_table = south_asia_table.sort_values("% of non-missing", ascending=False)

south_asia_table

In [ ]:
# filter only North America
north_america = country_stats[country_stats["Region"] == "North America"].copy()

# select columns to display
north_america_table = north_america[[
    "Country Name",
    "Country Code",
    "Income Group",
    "% of non-missing"
]]

# sort by data availability (highest first)
north_america_table = north_america_table.sort_values("% of non-missing", ascending=False)

north_america_table

In [ ]:
# filter only "Middle East, North Africa, Afghanistan & Pakistan"
mena_region = country_stats[
    country_stats["Region"] == "Middle East, North Africa, Afghanistan & Pakistan"
].copy()

# select columns to display
mena_table = mena_region[[
    "Country Name",
    "Country Code",
    "Income Group",
    "% of non-missing"
]]

# sort by data availability (highest first)
mena_table = mena_table.sort_values("% of non-missing", ascending=False)

mena_table

In [ ]:
# filter only Latin America & Caribbean
lac_region = country_stats[
    country_stats["Region"] == "Latin America & Caribbean"
].copy()

# select columns to display
lac_table = lac_region[[
    "Country Name",
    "Country Code",
    "Income Group",
    "% of non-missing"
]]

# sort by data availability (highest first)
lac_table = lac_table.sort_values("% of non-missing", ascending=False)

lac_table

In [ ]:
# filter only East Asia & Pacific
eap_region = country_stats[
    country_stats["Region"] == "East Asia & Pacific"
].copy()

# select columns to display
eap_table = eap_region[[
    "Country Name",
    "Country Code",
    "Income Group",
    "% of non-missing"
]]

# sort by data availability (highest first)
eap_table = eap_table.sort_values("% of non-missing", ascending=False)

eap_table

In [ ]:
# filter only Sub-Saharan Africa
ssa_region = country_stats[
    country_stats["Region"] == "Sub-Saharan Africa"
].copy()

# select columns to display
ssa_table = ssa_region[[
    "Country Name",
    "Country Code",
    "Income Group",
    "% of non-missing"
]]

# sort by data availability (highest first)
ssa_table = ssa_table.sort_values("% of non-missing", ascending=False)

ssa_table

In [ ]:
# filter only Europe & Centra Asia
eca_region = country_stats[
    country_stats["Region"] == "Europe & Central Asia"
].copy()

# select columns to display
eca_table = eca_region[[
    "Country Name",
    "Country Code",
    "Income Group",
    "% of non-missing"
]]

# sort by data availability (highest first)
eca_table = eca_table.sort_values("% of non-missing", ascending=False)

eca_table

In [ ]:
# --- list of the 50 selected countries ---

selected_countries = [
    # North America
    "United States", "Canada",
    
    # South Asia
    "India", "Sri Lanka", "Nepal", "Bangladesh", "Maldives", "Bhutan",
    
    # MENA + Afghanistan & Pakistan
    "Israel", "Iran, Islamic Rep.", "Egypt, Arab Rep.", "Tunisia",
    "Saudi Arabia", "Pakistan", "Algeria",
    
    # Latin America & Caribbean
    "Brazil", "Colombia", "Mexico", "Costa Rica", "Uruguay", "Chile",
    "Honduras", "Bolivia", "Dominican Republic", "Peru",
    
    # East Asia & Pacific
    "Japan", "Korea, Rep.", "Australia", "China", "Indonesia",
    "Viet Nam", "Philippines", "Cambodia",
    
    # Sub-Saharan Africa
    "South Africa", "Mauritius", "Nigeria", "Ghana", "Kenya",
    "Madagascar", "Rwanda", "Burkina Faso",
    
    # Europe & Central Asia
    "Germany", "France", "United Kingdom", "Poland", "Romania",
    "Hungary", "Georgia", "Kazakhstan", "Uzbekistan"
]

panel_50 = panel[panel["Country Name"].isin(selected_countries)].copy()
panel_50["Country Name"].nunique()

In [ ]:
# --- Table of the 50 chosen countries ---
countries_df = pd.DataFrame({"Country Name": selected_countries})

info_df = (
    countries_df
    .merge(
        panel[["Country Name", "Country Code", "Region", "Income Group"]].drop_duplicates(),
        on="Country Name",
        how="left"
    )
    .sort_values("Country Name")   # alphabetical sorting
    .reset_index(drop=True)
)

# make index start at 1 instead of 0
info_df.index = info_df.index + 1

# display final table
info_df

In [ ]:
# split the table in two parts
df_part1 = info_df.iloc[:25].copy()
df_part2 = info_df.iloc[25:].copy()

df_part1.index = df_part1.index
df_part2.index = df_part2.index

In [ ]:
# --- Use matplotlib to convert this table to a pdf ---

import matplotlib.pyplot as plt
import textwrap

def export_table_to_pdf(df, pdf_path, 
                        col_widths=None,
                        wrap_index=15,
                        wrap_cols=14,
                        figsize=(14, 12),
                        fontsize=11):

    # Copy data
    table = df.copy()
    n_rows, n_cols = table.shape

    # Wrap text
    def wrap_text(text, width):
        return "\n".join(textwrap.wrap(str(text), width=width))
    
    wrapped_index = [wrap_text(idx, wrap_index) for idx in table.index]
    wrapped_columns = [wrap_text(col, wrap_cols) for col in table.columns]

    # Default: equal width if not provided
    if col_widths is None:
        col_widths = [1 / n_cols] * n_cols

    # Build figure
    fig, ax = plt.subplots(figsize=figsize)
    ax.axis("off")

    tbl = ax.table(
        cellText=table.values,
        rowLabels=wrapped_index,
        colLabels=wrapped_columns,
        cellLoc="center",
        rowLoc="center",
        loc="center",
        colWidths=col_widths,
    )

    # Row heights
    for (row, col), cell in tbl.get_celld().items():
        if row == 0:
            cell.set_height(0.05)
        else:
            cell.set_height(0.06)

    tbl.auto_set_font_size(False)
    tbl.set_fontsize(fontsize)

    # Style cells
    for (row, col), cell in tbl.get_celld().items():
        cell.set_edgecolor("black")
        cell.set_linewidth(1)
        if row == 0:
            cell.set_text_props(weight="bold")
            cell.set_facecolor("#f5f5f5")

    plt.savefig(pdf_path, bbox_inches="tight")
    plt.close(fig)

    return pdf_path

export_table_to_pdf(
    df_part1,
    "sustainability-economic-performance/results/country_distribution/table_50_countries_part1.pdf",
    col_widths= [0.15, 0.13, 0.37, 0.20]
)

export_table_to_pdf(
    df_part2,
    "sustainability-economic-performance/results/country_distribution/table_50_countries_part2.pdf",
    col_widths= [0.15, 0.13, 0.37, 0.20]
)

In [ ]:
# display of the first five rows of the new dataset
panel_50.head()

In [ ]:
# what is the size of this new panel ?
panel_50.shape

In [ ]:
# --- Create a table showing the distribution of the sample of 50 countries ---

# extract country info (one row per country)
country_info = panel_50[["Country Name", "Region", "Income Group"]].drop_duplicates()

# crosstab: Region × Income Group with totals
region_income_50_table = pd.crosstab(
    country_info["Region"],
    country_info["Income Group"],
    margins=True,
    margins_name="Total"
)

# remove axis labels so "Region" and "Income Group" don't appear inside the table
region_income_50_table.index.name = None
region_income_50_table.columns.name = None

# styling helpers for Total row & Total column
def highlight_total_row(row):
    if row.name == "Total":
        return ['font-weight: bold; background-color: #f0f0f0'] * len(row)
    else:
        return [''] * len(row)

def highlight_total_col(col):
    if col.name == "Total":
        return ['font-weight: bold; background-color: #f0f0f0'] * len(col)
    else:
        return [''] * len(col)

# build a nicer styled table
styled_50_table = (
    region_income_50_table
    .style
    .set_table_styles([
        # whole table border behavior
        {
            "selector": "table",
            "props": [
                ("border-collapse", "collapse"),
                ("border", "1px solid #aaaaaa")
            ],
        },
        # header cells (top row: income groups)
        {
            "selector": "th.col_heading",
            "props": [
                ("background-color", "#f5f5f5"),
                ("font-weight", "bold"),
                ("border", "1px solid #aaaaaa"),
                ("padding", "6px"),
                ("text-align", "center")
            ],
        },
        # row labels (left side: regions + Total)
        {
            "selector": "th.row_heading",
            "props": [
                ("background-color", "#f5f5f5"),
                ("font-weight", "bold"),
                ("border", "1px solid #aaaaaa"),
                ("padding", "6px"),
                ("text-align", "left")
            ],
        },
        # corner cell (top-left)
        {
            "selector": "th.blank",
            "props": [
                ("background-color", "#f5f5f5"),
                ("border", "1px solid #aaaaaa")
            ],
        },
        # data cells
        {
            "selector": "td",
            "props": [
                ("border", "1px solid #cccccc"),
                ("padding", "6px"),
                ("text-align", "center")
            ],
        },
    ])
    .apply(highlight_total_row, axis=1)  # style Total row
    .apply(highlight_total_col, axis=0)  # style Total column
)

styled_50_table

In [ ]:
# --- Use matplotlib to convert this table to a pdf ---

import matplotlib.pyplot as plt
import textwrap

df = region_income_50_table.copy()
n_rows, n_cols = df.shape

# --- Wrap long labels ---

def wrap_text(text, width):
    """Insert line breaks every `width` characters (on nearest space)."""
    return "\n".join(textwrap.wrap(str(text), width=width))

# Wrap long region names (row index)
wrapped_index = [wrap_text(idx, width=23) for idx in df.index]

# Wrap long column headers (income groups)
wrapped_columns = [wrap_text(col, width=12) for col in df.columns]

# create the figure
fig, ax = plt.subplots(figsize=(15, 10))
ax.axis("off")

# build the table
tbl = ax.table(
    cellText=df.values,
    rowLabels=wrapped_index,      # regions + "Total"
    colLabels=wrapped_columns,    # income groups + "Total"
    cellLoc="center",
    rowLoc="center",
    loc="center",
    colWidths=[0.11] + [0.11] * (n_cols - 1),  # first col + others
)

# adjust cell heights
for (row, col), cell in tbl.get_celld().items():
    if row == 0:        # header row
        cell.set_height(0.06)
    else:               # body rows
        cell.set_height(0.07)

# 3. font size
tbl.auto_set_font_size(False)
tbl.set_fontsize(12)

# 4. style cells
for (row, col), cell in tbl.get_celld().items():
    # borders
    cell.set_edgecolor("black")
    cell.set_linewidth(1)

    # header row (income groups)
    if row == 0:
        cell.set_text_props(weight="bold")
        cell.set_facecolor("#f5f5f5")

    # row labels (regions + "Total") live in column -1 in matplotlib's table
    if col == -1:
        cell.set_text_props(weight="bold")
        cell.set_facecolor("#f5f5f5")

    # total column OR Total row in bold + light grey
    if col == n_cols - 1 or row == n_rows:
        cell.set_text_props(weight="bold")
        cell.set_facecolor("#f0f0f0")

plt.tight_layout()

# save to PDF under results/country_distribution
output_path = "sustainability-economic-performance/results/country_distribution/region_income_table_50.pdf"
plt.savefig(output_path, bbox_inches="tight")
plt.show()

output_path

In [ ]:
# save the final 50-country dataset
output_path = "sustainability-economic-performance/data/processed/panel_50_countries.csv"
panel_50.to_csv(output_path, index=False)

print("Saved to:", output_path)